In [ ]:
class Matrix:
    

In [ ]:
def stassen(A,B):
    n=len(A)

Learning
for stress testing closestpair finding
i used srand with differen seeds 0->10^6
to find the smallest n to find error I ran a loop form 2->10^6
if was z-=2,z<100;z++
k=0;k<10^6;k++
srand(k)
z size vector with random elements
run algo
find mismaqtch with bruteforce


In [ ]:
0 padding with induced if else
